In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/train.tsv
/kaggle/input/fake-news/valid.tsv


In [2]:
import nltk
import re
import string 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
from numpy import asarray
from numpy import zeros
import tensorflow as tf
from tensorflow import keras
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing import text,sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout

In [3]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [4]:
import keras_tuner as kt

In [5]:
train_data = pd.read_csv('../input/fake-news/train.tsv',sep="\t")

In [6]:
train_data.columns=['id','label','statement','subjects','speaker','speaker_job_title','state_info','party_affiliation','count_1','count_2','count_3','count_4','count_5','context']

In [7]:
for i in range(0,len(train_data)):
  if((train_data['label'][i]=='true')):
    train_data['label'][i]=1
  else:
    train_data['label'][i]=0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
data=train_data
data.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affiliation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,0,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [9]:
data['text']= data['statement'] + " " + data['subjects'] + " " + data['context']
del data['statement']
del data['id']
del data['subjects']
del data['context']
del data['speaker']
del data['speaker_job_title']
del data['state_info']
del data['party_affiliation']
del data['count_1']
del data['count_2']
del data['count_3']
del data['count_4']
del data['count_5']
data.head()

,label,text
0,0,When did the decline of coal start? It started...
1,0,"Hillary Clinton agrees with John McCain ""by vo..."
2,0,Health care reform legislation is likely to ma...
3,0,The economic turnaround started at the end of ...
4,1,The Chicago Bears have had more starting quart...


In [10]:
data.dropna()
data.fillna("", inplace=True)

In [11]:
pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=02993238402172d01e8bbf8f0e401bdea04cd300dd630064d0db17374ade3855
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [12]:
import nltk
nltk.download('punkt')
from bs4 import BeautifulSoup
nltk.download("stopwords")   
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
def remove_characters(text):
    return re.sub("[^a-zA-Z]"," ",text)

#Removal of stopwords 
def remove_stopwords_and_lemmatization(text):
    final_text = []
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    for word in text:
        if word not in set(stopwords.words('english')):
            lemma = nltk.WordNetLemmatizer()
            word = lemma.lemmatize(word) 
            final_text.append(word)
    return " ".join(final_text)

#Total function
def cleaning(text):
    text = remove_characters(text)
    text = remove_stopwords_and_lemmatization(text)
    return text

#Apply function on text column
data['text']=data['text'].apply(cleaning)

In [14]:
data=data.sample(frac=1)

**Training when data is 5% labled**

In [15]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.1 ,random_state=42)

In [16]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.80,random_state=0)

In [17]:
max_features = 3000
maxlen = 50

In [18]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [19]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [20]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [21]:
batch_size = 128

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model(hp):
    model = Sequential()
    neurons = hp.Choice('units', values=[32,64, 128,256])
    model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
    model.add(LSTM(units=neurons , return_sequences = True , recurrent_dropout =0.25,dropout=0.25))
    model.add(LSTM(units=neurons , recurrent_dropout = 0.25 , dropout = 0.25))
    model.add(Dense(units=neurons , activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
    model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=3)

2022-04-16 03:29:21.512855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 03:29:21.599390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 03:29:21.600202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-16 03:29:21.601528: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [24]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [25]:
tuner.search(X_lvl,y_lvl, epochs=20, validation_split=0.2, callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 10 Complete [00h 01m 37s]
val_accuracy: 0.8617886304855347

Best val_accuracy So Far: 0.8617886304855347
Total elapsed time: 00h 11m 25s


In [26]:
maxitr=3
itera=0
tl=len(data)-len(X_test)
ixll=len(X_lvl)
ixul=len(X_unl)
X_train=X_lvl
y_train=y_lvl
model=tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)
while(len(X_unl)>0):
    pred1=(model.predict(X_unl))
    pred0=1-pred1
    df_pred_prob = pd.DataFrame([])
    df_pred_prob['prob_0'] = (pred0).tolist()
    df_pred_prob['prob_1'] = (pred1).tolist()
    df_pred_prob['text']=X_unl.tolist()
    df=df_pred_prob
    yln=[]
    Xln=[]
    Xun=[]
    for i in range(len(X_unl)):
        if((df['prob_0'][i][0])>0.75):
            yln=yln+[0]
            Xln=Xln+[df['text'][i]]
        elif((df['prob_1'][i][0])>0.75):
            yln=yln+[1]
            Xln=Xln+[df['text'][i]]
        else:
            Xun=Xun+[df['text'][i]]
    
    y_train=np.concatenate((y_train,yln),axis=0)
    X_train=np.concatenate((X_train,Xln),axis=0)
    ixll=len(X_train)
    print("length of train data:=")
    print(ixll)
    model=tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, y_train, validation_split=0.3, epochs=50, batch_size=batch_size, shuffle=True, verbose = 1)
    X_unl = np.array(Xun)
    print("length of remaining unlabled data:=")
    print(len(X_unl))
    itera=itera+1
    if (itera==maxitr):
        break
    

Epoch 1/100
11/11 [==============================] - 9s 511ms/step - loss: 0.7622 - accuracy: 0.7829 - val_loss: 0.4277 - val_accuracy: 0.8499
Epoch 2/100
11/11 [==============================] - 5s 428ms/step - loss: 0.4698 - accuracy: 0.8326 - val_loss: 0.4394 - val_accuracy: 0.8499
Epoch 3/100
11/11 [==============================] - 4s 373ms/step - loss: 0.4587 - accuracy: 0.8326 - val_loss: 0.4435 - val_accuracy: 0.8499
Epoch 4/100
11/11 [==============================] - 5s 415ms/step - loss: 0.4634 - accuracy: 0.8326 - val_loss: 0.4229 - val_accuracy: 0.8499
Epoch 5/100
11/11 [==============================] - 4s 386ms/step - loss: 0.4588 - accuracy: 0.8326 - val_loss: 0.4241 - val_accuracy: 0.8499
Epoch 6/100
11/11 [==============================] - 4s 385ms/step - loss: 0.4590 - accuracy: 0.8326 - val_loss: 0.4259 - val_accuracy: 0.8499
Epoch 7/100
11/11 [==============================] - 5s 428ms/step - loss: 0.4535 - accuracy: 0.8326 - val_loss: 0.4231 - val_accuracy: 0.8499

In [27]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [28]:
count=0
for i in range(0,len(y_test_c)):
    if(y_test_c[i]==test_res[i]):
        count=count+1
print(count)

848


In [29]:
accuracy=(count/len(test_res))*100

In [30]:
print("Accuracy at 20% labled data",accuracy)

Accuracy at 20% labled data 82.8125


In [31]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.83      1.00      0.91       848
        Real       0.00      0.00      0.00       176

    accuracy                           0.83      1024
   macro avg       0.41      0.50      0.45      1024
weighted avg       0.69      0.83      0.75      1024



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
